# Healthcare Absence Management Pipeline

## Introduction

In large public healthcare systems, managing healthcare contracts and tracking personnel data, including absences, is a complex and time-consuming task. This project automates the process of handling absence reports and professional records within a public-private healthcare contract. By developing a robust data pipeline, this project seeks to minimize manual effort, reduce errors, and generate reliable reports to support decision-making.

The core objective of this notebook is to demonstrate a complete data processing pipeline, which includes:
- **Data Integration**: Merging absence reports with facility information to create a unified dataset.
- **Data Cleaning and Normalization**: Standardizing and cleaning the data to ensure consistency.
- **Business Rule Application**: Implementing business logic, such as reclassifying medical leave and handling special cases.
- **Data Consolidation**: Aggregating professional records and absences to create a comprehensive report.
- **Automated Report Generation**: Generating detailed absence and professional reports that facilitate compliance monitoring and contract management.

### Key Outcomes:
- Consolidated absence reports that simplify the analysis of employee absences.
- Professional reports with a new field `LEGAL_ABSENCES`, automating a previously manual process of validating and documenting absences.
- Insightful reports that can be used by managers to track and manage personnel more effectively.

This notebook provides a step-by-step implementation of the pipeline, with code, explanations, and visualizations to help understand each stage of the process. You can follow the notebook to see how raw data is transformed into actionable reports.

In [1]:
import pandas as pd
from unidecode import unidecode
from IPython.display import Markdown, display
from scripts.utils import (
    display_custom,
    rename_absence_report_columns,
    PROFESSIONALS_COLS, 
    AUSENCIAS_COLS,
    ABSENCE_REPORT_PATH, 
    FACILITY_ID_PATH,
    EXCLUDED_PROFESSIONALS_PATH, 
    CONSOLIDATION_FAILURES_PATH, 
    ABSENCES_CONSOLIDATED_PATH, 
    PROFESSIONALS_CONSOLIDATED_PATH,
)

absence_report = pd.read_excel(ABSENCE_REPORT_PATH)
facility_id_lookup = pd.read_csv(FACILITY_ID_PATH, encoding='ISO-8859-1')

pd.set_option('display.max_colwidth', None)
display(facility_id_lookup.head(3))

,rel_2141_CNES,rel_2141_UNIDADE
0,2027070,AMA PADRE MANOEL DA NOBREGA
1,2027070,UBS PADRE MANOEL DA NOBREGA
2,2029626,CAPS ADULTO III VILA MATILDE


---
### Data Transformation: Indexing healthcare facilities with a lookup table

One of the primary goals of this project is to integrate multiple datasets into a unified database. To achieve this, we standardize the column names in the `absence_report` by applying the prefix `rel_2141_` to each column. This naming convention helps track the origin of each piece of information once the data is consolidated.

In this step, we also merge the `absence_report` with the `facility_id_lookup` table. This table contains two key columns:
- `rel_2141_CNES`: The **National Health Establishment Code** (CNES), a unique identifier for each healthcare facility, which is not present in the absence report.
- `rel_2141_UNIDADE`: The name of the healthcare facility, with normalized text for consistency.

The merge is necessary because the facility names differ slightly across datasets, and only the CNES provides a reliable way to link records across databases.

In [2]:
absence_report = rename_absence_report_columns(absence_report)

# Normalize text fields by removing specific patterns, accents, and standardizing case and spacing
absence_report['rel_2141_CARGO'] = absence_report['rel_2141_CARGO'].str.replace(r' (I|II|III|-\s*HC|-\s*CER|-\s*PAI|-\s*SRT)$', '', regex=True)
absence_report['rel_2141_HORARIO'] = absence_report['rel_2141_HORARIO'].str.replace('_.*?_', '', regex=True).replace(' - APRENDIZ', '')
for col in ['rel_2141_NOME', 'rel_2141_CARGO', 'rel_2141_UNIDADE', 'rel_2141_HORARIO']:
    absence_report[col] = absence_report[col].astype(str).apply(lambda texto: ' '.join(unidecode(texto).strip().upper().split()))

# Reduce the number of STATUS categories by mapping similar statuses to common values
status_map = {
    'Férias': 'Ativo',
    'Aviso Prévio': 'Ativo',
    'Af.Ac.Trabalho': 'Af.Previdência',
    'Doença Ocupacional': 'Af.Previdência',
    'Licença Mater.': 'Af.Previdência'
}
absence_report['rel_2141_STATUS'] = absence_report['rel_2141_STATUS'].replace(status_map)

absence_report = absence_report.merge(facility_id_lookup, on='rel_2141_UNIDADE', how='left')
absence_report = absence_report[AUSENCIAS_COLS]
absence_report.info()
display_custom(absence_report.head(3), class_name="custom_df")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2788 entries, 0 to 2787
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   rel_2141_CNES         2725 non-null   float64       
 1   rel_2141_UNIDADE      2788 non-null   object        
 2   rel_2141_NOME         2788 non-null   object        
 3   rel_2141_CARGO        2788 non-null   object        
 4   rel_2141_DT_ADMISSAO  2788 non-null   datetime64[ns]
 5   rel_2141_DT_DEMISSAO  93 non-null     object        
 6   rel_2141_prof_ID      2788 non-null   int64         
 7   rel_2141_HORARIO      2788 non-null   object        
 8   rel_2141_JORNADA      2788 non-null   int64         
 9   rel_2141_STATUS       2788 non-null   object        
 10  rel_2141_LICINICIO    1418 non-null   datetime64[ns]
 11  rel_2141_LICFIM       1359 non-null   datetime64[ns]
 12  rel_2141_AUSENCIA     2788 non-null   object        
dtypes: datetime64[ns](

,rel_2141_CNES,rel_2141_UNIDADE,rel_2141_NOME,rel_2141_CARGO,rel_2141_DT_ADMISSAO,rel_2141_DT_DEMISSAO,rel_2141_prof_ID,rel_2141_HORARIO,rel_2141_JORNADA,rel_2141_STATUS,rel_2141_LICINICIO,rel_2141_LICFIM,rel_2141_AUSENCIA
0,2786729.0,UBS A. E. DE CARVALHO,NOAMI MAFFUCCI,AGENTE COMUNITARIO DE SAUDE,2009-01-19,NaN,40000013,08:00 - 12:00 - 13:00 - 17:00 (SEG A SEX),40,Ativo,2024-08-15,2024-08-15,Atestado
1,2786729.0,UBS A. E. DE CARVALHO,DAMITA CUMMINE,AGENTE COMUNITARIO DE SAUDE,2008-04-22,NaN,40000015,08:00 - 12:00 - 13:00 - 17:00 (SEG A SEX),40,Ativo,NaT,NaT,Ativo
2,2787776.0,UBS JARDIM SAO FRANCISCO,SENA BEVER,AUXILIAR DE SAUDE BUCAL,2008-09-22,NaN,40000016,10:00 - 13:00 - 14:00 - 19:00 (SEG A SEX),40,Ativo,2024-08-16,2024-08-20,Atestado


### Absence Report Overview

This dataset contains a comprehensive list of professionals currently employed by the social organization responsible for managing public health policies in a territory of approximately 500,000 inhabitants. The data includes not only the absences of these professionals but also the full roster of employees, allowing us to generate two reports: one with the complete list of professionals, and another focused solely on absences.

- `rel_2141_CARGO`: Contains the job category or professional role of the employee.
- `rel_2141_DT_ADMISSAO`: Date of admission or hiring.
- `rel_2141_DT_DEMISSAO`: Date of termination or dismissal.
- `rel_2141_prof_ID`: A unique identifier assigned to each employee by the organization.
- `rel_2141_HORARIO`: The employee's work schedule.
- `rel_2141_JORNADA`: The number of contracted hours per week.
- `rel_2141_STATUS`: The employee's current status at the time the report was generated (e.g., Active, On Vacation, Terminated, etc.).
- `rel_2141_LICINICIO`: Start date of a leave or absence.
- `rel_2141_LICFIM`: End date of a leave, when applicable.
- `rel_2141_AUSENCIA`: The type of absence (e.g., Vacation, Medical Leave, Maternity Leave).

It is important to note that the `rel_2141_CNES` column was converted to float due to null values. Additionally, the termination date (`rel_2141_DT_DEMISSAO`) column will be converted to the proper datetime format:

In [3]:
for col in ['rel_2141_DT_ADMISSAO', 'rel_2141_DT_DEMISSAO', 'rel_2141_LICINICIO', 'rel_2141_LICFIM']:
    absence_report[col] = pd.to_datetime(absence_report[col], dayfirst=True, errors='coerce')

display(Markdown(f"##### The data type of 'rel_2141_DT_DEMISSAO' now is **{absence_report['rel_2141_DT_DEMISSAO'].dtype}**"))

##### The data type of 'rel_2141_DT_DEMISSAO' now is **datetime64[ns]**

---
### Excluding Non-Eligible Professionals

In this step, we filter out professionals who should not be included in the current absence report for various reasons. After applying these exclusion criteria, we generate a separate report of the excluded professionals for further review, which is saved as an csv file. This report is useful for understanding who was excluded and why, ensuring transparency and traceability in the data processing.

In [4]:
status_exclusion_criteria  = absence_report['rel_2141_STATUS'].isin([
    'Apos. por Incapacidade Permanente',  # 'Retired due to Permanent Disability'
    'Licença s/venc',  # 'Unpaid Leave'
    'Contrato de Trabalho Suspenso',  # 'Employment Contract Suspended'
    'Admissão prox.mês',  # 'Hired for Next Month'
])
display(absence_report[status_exclusion_criteria ]['rel_2141_STATUS'].value_counts().reset_index(name='COUNT').rename(columns={'rel_2141_STATUS': 'STATUS'}))

# Exclude professionals without a CNES, typically administrative staff not part of the contract
cnes_missing_criteria = pd.isna(absence_report['rel_2141_CNES'])
display(absence_report[cnes_missing_criteria]['rel_2141_UNIDADE'].value_counts().reset_index(name='COUNT').rename(columns={'rel_2141_UNIDADE': 'FACILITY'}))

# Save excluded professionals to a separate report for further review
excluded_professionals = absence_report[status_exclusion_criteria  | cnes_missing_criteria]
excluded_professionals.sort_values(by=['rel_2141_CNES', 'rel_2141_STATUS', 'rel_2141_NOME']).to_csv(EXCLUDED_PROFESSIONALS_PATH, index=False, encoding='ISO-8859-1')

# After excluding non-eligible professionals, the absence and professional DataFrames are created and ready for transformation.
absence_report = absence_report[~(status_exclusion_criteria | cnes_missing_criteria)]
absence_report['rel_2141_CNES'] = absence_report['rel_2141_CNES'].astype('Int64')
professional_report_initial = absence_report[PROFESSIONALS_COLS]
professional_report_initial = professional_report_initial.sort_values(by=['rel_2141_CNES', 'rel_2141_CARGO', 'rel_2141_JORNADA', 'rel_2141_NOME'])

,STATUS,COUNT
0,Admissão prox.mês,37
1,Apos. por Incapacidade Permanente,5
2,Licença s/venc,2
3,Contrato de Trabalho Suspenso,2


,FACILITY,COUNT
0,COORDENACAO TECNICA ADMINISTRATIVA - SAS,63


### Data Consolidation: Professional Records

In this step, we consolidate the records of professionals based on their unique identifier (`rel_2141_prof_ID`). The goal is to ensure that each professional has a single, consistent record in the final dataset, while identifying and handling any discrepancies in the data.

In [5]:
def consolidate_professional(group):
    # Check if there is only one unique termination date and apply it to all records in the group
    demissao_unique = group['rel_2141_DT_DEMISSAO'].dropna().unique()
    if len(demissao_unique) == 1:
        group['rel_2141_DT_DEMISSAO'] = group['rel_2141_DT_DEMISSAO'].fillna(demissao_unique[0])

    # Ensure data integrity by keeping only groups with unique values for all columns
    if any(group[col].nunique(dropna=False) > 1 for col in group.columns):
        return None
    
    return group.iloc[0]

# Group professionals by their unique ID and consolidate records into a single row if no inconsistencies are found
professional_report = professional_report_initial.groupby('rel_2141_prof_ID')[PROFESSIONALS_COLS].apply(consolidate_professional).dropna(how='all')
professional_report[['rel_2141_CNES', 'rel_2141_prof_ID', 'rel_2141_JORNADA']] = professional_report[['rel_2141_CNES', 'rel_2141_prof_ID', 'rel_2141_JORNADA']].astype('Int64')

invalid_groups = professional_report_initial.loc[~professional_report_initial['rel_2141_prof_ID'].isin(professional_report.index)]
invalid_groups.to_csv(CONSOLIDATION_FAILURES_PATH, index=False, encoding='ISO-8859-1')
display_custom(invalid_groups, class_name="custom_df")


,rel_2141_CNES,rel_2141_UNIDADE,rel_2141_NOME,rel_2141_CARGO,rel_2141_DT_ADMISSAO,rel_2141_DT_DEMISSAO,rel_2141_prof_ID,rel_2141_HORARIO,rel_2141_JORNADA,rel_2141_STATUS
127,2787806,UBS JARDIM SAO NICOLAU,MILDRID HEBBORN,AUXILIAR ADMINISTRATIVO,2009-03-19,2024-08-19,40000351,10:00 - 13:00 - 14:00 - 19:00 (SEG A SEX),40,Demitido
128,2787806,UBS JARDIM SAO NICOLAU,MILDRID HEBBORN,AUXILIAR ADMINISTRATIVO,2009-03-19,2024-08-20,40000351,10:00 - 13:00 - 14:00 - 19:00 (SEG A SEX),40,Demitido


---
### Business Rules Processing for Absences

This block applies key business rules to clean and correct inconsistencies in the absence data:

1. **Initial Data Cleaning**: We remove records that do not represent actual absences. First, we exclude entries with no absence start date, followed by the removal of records indicating termination (`Demitido`, `Demitidos`, `Aviso Prévio`).

2. **Inconsistency Correction**: A widespread issue in the dataset is the inconsistent classification of medical leave. Absences under 15 days are sometimes incorrectly marked as "Af.Previdência" (Social Security leave) instead of "Atestado" (Medical Certificate). We correct this by reclassifying those under 15 days as "Atestado."

3. **Manual Review Flagging**: Medical certificates that extend beyond 15 days are flagged for manual review, as these should potentially be classified as Social Security leave. To highlight these cases, the absence type is updated to "Atestado (OVER 15 DAYS)."

In [6]:
# 1. Initial Data Cleaning
initial_count = absence_report.shape[0]
absence_report = absence_report[
    ~pd.isna(absence_report['rel_2141_LICINICIO']) & 
    ~absence_report['rel_2141_AUSENCIA'].isin(['Demitido', 'Demitidos', 'Aviso Prévio'])
]
excluded_rows = initial_count - absence_report.shape[0]
# display(f"Number of rows excluded: {excluded_rows} (out of {initial_count})")
display(Markdown(f"#### - Number of rows excluded: **{excluded_rows} _(out of {initial_count})_**"))


# 2. Inconsistency Correction
social_security_filter = (
    (absence_report['rel_2141_AUSENCIA'] == 'Af.Previdência') &
    ~pd.isna(absence_report['rel_2141_LICINICIO']) &
    ~pd.isna(absence_report['rel_2141_LICFIM'])
)

# Calculate the duration of each absence and make the correction
absence_duration = (absence_report['rel_2141_LICFIM'] - absence_report['rel_2141_LICINICIO']).dt.days
altered_rows = (social_security_filter & (absence_duration < 15)).sum()
absence_report.loc[social_security_filter & (absence_duration < 15), 'rel_2141_AUSENCIA'] = 'Atestado'

total_social_security = social_security_filter.sum()
altered_percentage = (altered_rows / total_social_security) * 100 if total_social_security > 0 else 0
display(Markdown(f"#### - Percentage of rows changed from social security leave to regular medical leave: **{altered_percentage:.2f}% ({altered_rows}/{total_social_security} filtered)**"))

# 3. Manual Review Flagging
medical_certificate_filter = (
    (absence_report['rel_2141_AUSENCIA'] == 'Atestado') &
    ~pd.isna(absence_report['rel_2141_LICINICIO']) &
    ~pd.isna(absence_report['rel_2141_LICFIM'])
)

# Flagging certificates longer than 15 days
absence_report.loc[medical_certificate_filter & (absence_duration > 15), 'rel_2141_AUSENCIA'] = 'Atestado (Acima de 15 dias)'

# Get professionals flagged for manual review
professionals_with_manual_review = absence_report.loc[
    absence_report['rel_2141_AUSENCIA'] == 'Atestado (Acima de 15 dias)', 'rel_2141_prof_ID'
].unique()

manual_review_filter = absence_report['rel_2141_prof_ID'].isin(professionals_with_manual_review)
display(Markdown(f"#### - Number of rows flagged for manual review (usually maternity leave): **{manual_review_filter.sum()}**"))
display_custom(absence_report.loc[manual_review_filter].head(4), class_name="custom_df")

# Export the consolidated absence report after the initial filtering
absence_report = absence_report.sort_values(by=['rel_2141_prof_ID', 'rel_2141_LICINICIO', 'rel_2141_LICFIM'])
absence_report.to_csv(ABSENCES_CONSOLIDATED_PATH, index=False, encoding='ISO-8859-1')

#### - Number of rows excluded: **1321 _(out of 2680)_**

#### - Percentage of rows changed from social security leave to regular medical leave: **98.77% (641/649 filtered)**

#### - Number of rows flagged for manual review (usually maternity leave): **8**

,rel_2141_CNES,rel_2141_UNIDADE,rel_2141_NOME,rel_2141_CARGO,rel_2141_DT_ADMISSAO,rel_2141_DT_DEMISSAO,rel_2141_prof_ID,rel_2141_HORARIO,rel_2141_JORNADA,rel_2141_STATUS,rel_2141_LICINICIO,rel_2141_LICFIM,rel_2141_AUSENCIA
525,2788861,UBS VILA ESPERANCA - DR. CASSIO BITTENCOURT FILHO,BORD CROYSER,ENFERMEIRO ESF,2016-07-04,NaT,40001562,06:30 - 11:00 - 12:00 - 15:30 (SEG A SEX),40,Af.Previdência,2024-08-01,2025-05-27,Atestado (Acima de 15 dias)
1239,2788926,UBS VILA GRANADA - DR. ALFREDO FERREIRA PAULINO FILHO P.A.I,LORENZO WELDS,AUXILIAR DE ENFERMAGEM,2021-09-01,NaT,40004395,07:30 - 12:00 - 13:00 - 16:30 (SEG A SEX),40,Af.Previdência,2024-08-20,2025-06-15,Atestado (Acima de 15 dias)
1486,2752204,UBS CANGAIBA - DR. CARLOS GENTILE DE MELLO,SPENCER KERSHAW,AUXILIAR DE ENFERMAGEM,2021-12-13,NaT,40004975,12:45 - 16:00 - 16:15 - 19:00 (SEG A SAB),36,Af.Previdência,2024-05-02,2024-08-29,Licença Mater.
1487,2752204,UBS CANGAIBA - DR. CARLOS GENTILE DE MELLO,SPENCER KERSHAW,AUXILIAR DE ENFERMAGEM,2021-12-13,NaT,40004975,12:45 - 16:00 - 16:15 - 19:00 (SEG A SAB),36,Af.Previdência,2024-08-30,2024-11-02,Atestado (Acima de 15 dias)


### Consolidating Uninterrupted Absences

To streamline the analysis of employee absences, we consolidate uninterrupted periods of the same type of absence. This means that if an employee has consecutive days of the same absence type (e.g., continuous medical leave), these are combined into a single entry. This process simplifies the data, making it easier to analyze and interpret the length and frequency of absences without redundancy.

In [7]:
def consolidate_absence(group):
    # Initialize control variables
    absence_start = None
    absence_end = None
    current_status = None
    consolidated = []

    for idx, row in group.iterrows():
        if absence_start is None:
            # Set the initial values for the first row
            absence_start = row['rel_2141_LICINICIO']
            absence_end = row['rel_2141_LICFIM']
            current_status = row['rel_2141_AUSENCIA']
        else:
            # Check for continuity and if the absence type is the same
            if row['rel_2141_LICINICIO'] == absence_end + pd.Timedelta(days=1) and row['rel_2141_AUSENCIA'] == current_status:
                # Extend the end date if the absence is continuous and the status is the same
                absence_end = row['rel_2141_LICFIM']
            else:
                # Append the consolidated interval
                consolidated.append({
                    'rel_2141_prof_ID': row['rel_2141_prof_ID'],
                    'rel_2141_LICINICIO': absence_start,
                    'rel_2141_LICFIM': absence_end,
                    'rel_2141_AUSENCIA': current_status
                })
                # Update to a new interval
                absence_start = row['rel_2141_LICINICIO']
                absence_end = row['rel_2141_LICFIM']
                current_status = row['rel_2141_AUSENCIA']

    # Append the last consolidated interval
    consolidated.append({
        'rel_2141_prof_ID': row['rel_2141_prof_ID'],
        'rel_2141_LICINICIO': absence_start,
        'rel_2141_LICFIM': absence_end,
        'rel_2141_AUSENCIA': current_status
    })

    return pd.DataFrame(consolidated)

# display("Absences of a single employee before consolidation:")
display(Markdown("#### Absences of a single employee before consolidation:"))
display(absence_report[['rel_2141_prof_ID', 'rel_2141_LICINICIO', 'rel_2141_LICFIM', 'rel_2141_AUSENCIA']][absence_report['rel_2141_prof_ID'] == 40002244])

absence_report = absence_report.groupby('rel_2141_prof_ID')[AUSENCIAS_COLS].apply(consolidate_absence).reset_index(drop=True)

display(Markdown("#### Absences after consolidation:"))
display(absence_report[absence_report['rel_2141_prof_ID'] == 40002244])

#### Absences of a single employee before consolidation:

,rel_2141_prof_ID,rel_2141_LICINICIO,rel_2141_LICFIM,rel_2141_AUSENCIA
653,40002244,2024-08-02,2024-08-02,Atestado
654,40002244,2024-08-03,2024-08-03,Atestado
655,40002244,2024-08-05,2024-08-05,Atestado
656,40002244,2024-08-08,2024-08-08,Atestado
657,40002244,2024-08-12,2024-08-12,Atestado
658,40002244,2024-08-13,2024-08-13,Atestado
659,40002244,2024-08-23,2024-08-23,Atestado


#### Absences after consolidation:

,rel_2141_prof_ID,rel_2141_LICINICIO,rel_2141_LICFIM,rel_2141_AUSENCIA
318,40002244,2024-08-02,2024-08-03,Atestado
319,40002244,2024-08-05,2024-08-05,Atestado
320,40002244,2024-08-08,2024-08-08,Atestado
321,40002244,2024-08-12,2024-08-13,Atestado
322,40002244,2024-08-23,2024-08-23,Atestado


---
### Automating the Generation of Legal Absences

One of the key motivations for generating the `LEGAL_ABSENCES` column is to streamline the documentation process and automate the manual verification of absence data for each professional.

Previously, each healthcare unit manager was responsible for manually compiling the legal absences for their staff, which resulted in a high number of errors and inconsistencies. By centralizing and automating this process through the pipeline, we not only reduce manual effort but also ensure that the absence data is accurate and standardized across the entire organization.

Below is a table showing a selection of manually generated records, which highlight how inefficient and error-prone the previous manual process was in January, 2024. The automation processes developed throughout the contract are only made possible through this type of pipeline, as it allows for vectorized operations where business rules can be applied automatically across the entire dataset:
  
> Consultório na rua - ABONO GESTACIONAL DE 20/12 A 25/12 E 26/12 LICENÇA MATERNIDADE\nLicença Mater. 26/12/2023 - 20/10/2024  
AMA CANGAÍBA - ATIVIDADE TEÓRICA DE 05/01; 12/01; 19/01; 26/01/2024  
UBS CANGAÍBA - Licença gestante\nAtestado médico de 15 a 15/01 - 2 dias\nAtestado médico de 18 a 30/01 - 13 dias  
ATESTADO - 03/01/24; ATESTADO - 15 A 18/01/24\nATESTADO 31/01/2024  
FÉRIAS - 03 A 22/01/24, FOGA SAÚDE - 23/01/24  
Atestado 04.01 a 06.01 / 17 e 18.01 / 24 a 26.01 / 29.01  
AFASTAMENTO GESTACIONAL EM 04/09 PREVISÃO DE PARTO PARA 17/04/2024\nLicença Mater. 04/09/2023 - 29/06/2024  
11/01 e 12/01 3h BH; 15/01 BH 8h\nFérias 31/01 a 09/02/2024\nInércia Clínica no Risco Cardiovascular  
FERIAS 19/12/2023 A 02/01/2024 (15 DIAS)\nATESTADO 17/01/2024 A 19/01/2024 (3 DIAS)\nAFASTAMENTO GESTACIONAL COM INÍCIO EM 22/01/2024 A 16/11/2024 (DPP 20/09/2024)  
Atestado 26/01/2024 - 30/01/2024


In [8]:
def generate_legal_absences(prof_row):
    # Identify if the professional has any absences to process
    absences = absence_report[absence_report['rel_2141_prof_ID'] == prof_row['rel_2141_prof_ID']]
    if absences.empty:
        return pd.NA
    
    absences_text = []
    
    # Iterate through the professional's absences
    for _, absence_row in absences.iterrows():
        status = absence_row['rel_2141_AUSENCIA']
        start_date = absence_row['rel_2141_LICINICIO'].strftime('%d/%m/%Y')
        end_date = absence_row['rel_2141_LICFIM']

        # Generate text based on the presence or absence of end date (LICFIM)
        if pd.isna(end_date):  # An absence without a return date
            text = f"{status}: A partir de {start_date}"  # 'STARTING ON'
        elif start_date == end_date.strftime('%d/%m/%Y'):  # A single day absence is simplified
            text = f"{status}: {start_date}"
        else:  # 2+ days absence with an end date
            end_date_formatted = end_date.strftime('%d/%m/%Y')
            text = f"{status}: {start_date} a {end_date_formatted}"  # start_date 'TO' end_date
        
        absences_text.append(text)
    
    return ' | '.join(absences_text)

# Apply the generate_legal_absences function to the professional report DataFrame
professional_report['LEGAL_ABSENCES'] = professional_report.apply(generate_legal_absences, axis=1)

# Final result
professional_report.to_csv(PROFESSIONALS_CONSOLIDATED_PATH, index=False, encoding='ISO-8859-1')

display(professional_report[['LEGAL_ABSENCES']].dropna().head(10))
professional_report.info()

,LEGAL_ABSENCES
rel_2141_prof_ID,
40000013,Atestado: 15/08/2024
40000016,Atestado: 16/08/2024 a 20/08/2024
40000023,Férias: 10/07/2024 a 08/08/2024 | Atestado: 09/08/2024 a 22/08/2024
40000025,Atestado: 20/08/2024 a 24/08/2024
40000028,Atestado: 04/09/2024
40000039,Af.Previdência: A partir de 04/07/2023
40000056,Férias: 05/08/2024 a 03/09/2024
40000057,Atestado: 09/08/2024 a 13/08/2024
40000069,Atestado: 27/08/2024 a 31/08/2024


<class 'pandas.core.frame.DataFrame'>
Index: 2226 entries, 40000013 to 40006729
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   rel_2141_CNES         2226 non-null   Int64         
 1   rel_2141_UNIDADE      2226 non-null   object        
 2   rel_2141_NOME         2226 non-null   object        
 3   rel_2141_CARGO        2226 non-null   object        
 4   rel_2141_DT_ADMISSAO  2226 non-null   datetime64[ns]
 5   rel_2141_DT_DEMISSAO  89 non-null     datetime64[ns]
 6   rel_2141_prof_ID      2226 non-null   Int64         
 7   rel_2141_HORARIO      2226 non-null   object        
 8   rel_2141_JORNADA      2226 non-null   Int64         
 9   rel_2141_STATUS       2226 non-null   object        
 10  LEGAL_ABSENCES        944 non-null    object        
dtypes: Int64(3), datetime64[ns](2), object(6)
memory usage: 215.2+ KB
